In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
df = pd.read_csv('/content/station_data_dataverse3.csv')

In [ ]:
label_encoder = LabelEncoder()
df['weekday'] = label_encoder.fit_transform(df['weekday'])
df['platform'] = label_encoder.fit_transform(df['platform'])


In [ ]:
df['distance'] = df['distance'].fillna(df['distance'].mean())

In [ ]:
X = df[
    ['chargeTimeHrs', 'startTime', 'endTime', 'weekday', 'distance', 'managerVehicle'] +
    ['userId', 'stationId', 'locationId']
]
y = df['kwhTotal']


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)


In [ ]:
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dropout(0.2),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 39.3568
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31.6658
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 18.3716
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.0563
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.3937
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.8541
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.7924
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.7483
Epoch 9/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.3715
Epoch 10/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.0196
Epoch 11/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.5609
Epoch 12/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.6815
Epoch 13/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.0743
Epoch 14/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.5047
Epoch 15/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7776
Epoch 16/50
85/

In [ ]:
y_pred = model.predict(X_test_lstm)

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [ ]:
print("RMSE:", rmse)
print("R² Score:", r2)
# print("MSE:", mse)
print("MAE:",mean_absolute_error(y_test, y_pred))
print("MAPE:",mean_absolute_percentage_error(y_test, y_pred))

RMSE: 2.1604407011285924
R² Score: 0.5131966839128177
MAE: 1.3371772449813641
MAPE: 33159648046857.336
